In [1]:
from song_scraper import song_scraper
import pandas as pd

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("display.max_columns", None)
pd.set_option("max_colwidth", 800)
pd.set_option("display.max_colwidth", None)

In [9]:
artists = ["Kwiat Jabłoni"]

In [10]:
df = song_scraper(artists)

c:\Users\wilko\Desktop\Studia\3 rok\Projekty studia\Python-Projekt\song_scraper.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  song_page = requests.get(row[0])
c:\Users\wilko\Desktop\Studia\3 rok\Projekty studia\Python-Projekt\song_scraper.py:74: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  re.findall(r",[^,]+,([^\.]+)\.", row[0])[0]


In [11]:
df.head()

,Author,Title,Polish,English
0,Kwiat Jabłoni,Hot16Challenge2,"To jest nasz pierwszy raz w życiu,\nNigdy nie było tak jeszcze,\nTak, że oboje piszemy jeden tekst.\nJuż dwa miesiące w ukryciu,\nTen sam jest obiad i przestrzeń.\nDzielimy te same myśli, więc słowa też.\nKażdego dnia, każdej nocy\nTa jedna myśl, a nie tysiąc:\nCo byłoby gdyby nie było tego, co było.\nNie mamy specjalnych mocy,\nAle możemy przysiąc,\nŻe tak na życie się wcześniej nie patrzyło.\n\nMamy tylko tu i teraz.\nBrakuje słów, zaczniemy od zera.\nTu i teraz.\nBrakuje słów, zaczniemy od zera.\nTu i teraz.\nBrakuje słów, zaczniemy od zera.\n\nNominujemy Jacka Stachurskiego,\nJurka Owsiaka, Sorry Boys, naszego Starego.\nJacka Stachurskiego,\nJurka Owsiaka, Sorry Boys, naszego Starego.","That's the first time in our lives\nIt never happened before\nThat the both of us are writing the same lyrics\nTwo months, already, we're hidden\nIt's the same dinner and and same space\nSharing the same thoughts, our words, too, are the same\nEvery single day and night\nThat single thought, not a thousand\nWhat would've happened if what happened hadn't have happened\nWe don't have special powers\nBut we can promise you this\nThis isn't how one used to look at life before now\n\nWe've only got here and now\nFor the lack of words, we'll start from scratch\nHere and now\nFor the lack of words, we'll start from scratch\nHere and now\nFor the lack of words, we'll start from scratch\n\nWe nominate Jacek Stachursky,\nJurek Owsiak, Sorry Boys, our Old Man.\nJacek Stachursky,\nJurek Owsiak, Sorry Boys, our Old Man."
1,Kwiat Jabłoni,Bankiet,"To tylko kolejny uścisk dłoni\r\nPod stołem by nikt nie widział\r\nTo tylko kolejne słowa\r\nPadły by nikt nie usłyszał\n\r\nA Ty jesteś spokojny i tak\r\nJak zawsze od lat na raz\r\nStajesz się wolny na dwa\r\nOślepniesz na świat\n\r\nJesteś kolejną twarzą\r\nDaleko na końcu bankietu\r\nPłyniesz cicho jak rzeka\r\nZ rozpędu trafiasz do celu\n\r\n[Ref]\r\nTylko nie przegap dnia\r\nJak gwiazdy spadające\r\nI nie przegap dnia\r\nZnika na horyzoncie\n\r\nNie przegap dnia\r\nJak gwiazdy spadające\r\nI nie przegap dnia\r\nZnika na horyzoncie\n\r\nMówisz, że takie są czasy\r\nI od nas nic nie zależy\r\nŻadne z nas nic nie znaczy\r\nKażdy ruch w pustkę uderzy\n\r\nA Ty jesteś spokojny i tak\r\nJak zawsze od lat na raz\r\nStaniesz się wolny na dwa\r\nOślepniesz na świat\n\r\nJesteś kolejną twarzą\r\nDaleko na końcu bankietu\r\nPłyniesz cicho jak rzeka\r\nZ rozpędu trafiasz do celu\n\r\n[Ref]\r\nTylko nie przegap dnia\r\nJak gwiazdy spadające\r\nI nie przegap dnia\r\nZnika na horyzoncie\n\r\nNie przegap dnia\r\nJak gwiazdy spadające\r\nI nie przegap dnia\r\nZnika na horyzoncie\n\r\nNie przegap dnia\r\nJak gwiazdy spadające\r\nI nie przegap dnia\r\nZnika na horyzoncie\n\r\nNie przegap dnia\r\nJak gwiazdy spadające\r\nI nie przegap dnia\r\nZnika na horyzoncie","[Title: The Banquet]\nIt's just another handshake\nUnder the table so that nobody sees it\nIt's just another words\nSpoken so that nobody hears it\n\nAnd you compose yourself anyway\nOn one, as you've always done before\nFreeing yourself, on two\nLosing sight of what's around you\n\nYou're just another face\nDeep at the back of the banquet\nFlowing silently like a river\nThe momentum brings you to your target\n\n[Chorus]\nJust don't let the day slip through your fingers\nLike the falling stars\nAnd don't let the day slip through your fingers\nIt's disappearing on the horizon\n\nDon't let the day slip through your fingers\nLike the falling stars\nAnd don't let the day slip through your fingers\nIt's disappearing on the horizon\n\nYou're saying it's the times we're living in\nIt's all out of our control\nWe don't have any meaning\nAll our struggles will come to nothing\n\nAnd you compose yourself anyway\nOn one, as you've always done before\nFreeing yourself, on two\nLosing sight of what's around you\n\nYou're just another face\nDeep at the back of the banquet\nFlowing silently like a river\nThe momentum br

In [12]:
df.duplicated().sum()

0